In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

import scipy as sc

import sklearn
from sklearn import feature_selection, datasets, model_selection, preprocessing, decomposition, metrics
from sklearn.model_selection import validation_curve, learning_curve, cross_validate, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

pd.set_option('display.max_columns', None)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import itertools
from sklearn.metrics import confusion_matrix
from python_utils import *
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '../../Libraries_Paper/libraries/Osc_libraries')
import utils

from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings('ignore')

import scipy.io as sio
import numpy
from sklearn.ensemble import RandomForestClassifier

## I check that the data arrays meet kernel condition....

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import scipy.io as sio

def normData (X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

def normData_minmax (X_train, X_test):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test


def is_pos_def(A):
    M = np.matrix(A)
    return np.all(np.linalg.eigvals(M+M.transpose()) > 0)

def matrixIsKernel(X_train, X_test):
    print("Without normalizing the kernel. Data entered to TCK already normalized. \n Out meets the kernel condition?: ")
    print(is_pos_def(X_train))

    print("Normalizing the kernel. Data entered to TCK already normalized. \n Out meets the kernel condition?: ")
    X_train, X_test = normData(X_train, X_test)
    print(is_pos_def(X_train))
    
    print("Normalizing the kernel. Data entered to TCK already normalized. \n Out meets the kernel condition?: ")
    X_train, X_test = normData_minmax(X_train, X_test)
    print(is_pos_def(X_train))

    
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.loc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_curve, auc
    
def nuSVM(params, folders, dependencia, sigma, random_state, debug):

    matrix_all_values = np.zeros((16, len(folders)))

    for i in range(len(folders)):
        
        print()
        print("###################### RESULTS [SVM] FOLDER " + folders[i] + " ######################")
        print()

        X_pre_train = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_train_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")
        X_test = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_test_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")

        y_pre_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_train_tensor.csv')
        y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_test_tensor.csv')

        semillas = [9,18,35,52,75]


        kf = KFold(n_splits=5, shuffle=False)
        kf.get_n_splits(X_pre_train)

        bestHyperparameters = {'nu': 0}

        bestMetricDev = 0
        for c in range(len(params['nu'])):
            clf = svm.NuSVC(kernel="precomputed", nu=params['nu'][c], probability=True, random_state=random_state)

            roc_auc_score = []
            threshold_1 = []
            threshold_2 = 0
            y_pred_arr = []
            # Divide the test set into k partitions
            for train_index, val_index in kf.split(X_pre_train):

                X_train, X_val = X_pre_train.iloc[train_index,train_index].reset_index(drop=True), X_pre_train.iloc[val_index,train_index].reset_index(drop=True)
                y_train, y_val = y_pre_train.iloc[train_index].reset_index(drop=True), y_pre_train.iloc[val_index].reset_index(drop=True)

                clf = clf.fit(X_train, y_train)
                y_pred = clf.predict_proba(X_val)[:,1]

                threshold_1.append(Find_Optimal_Cutoff(y_val, y_pred))
                auc = sklearn.metrics.roc_auc_score(y_val, y_pred)
                roc_auc_score.append(auc)

                y_pred_arr.append(y_pred)


            # Get the threshold for: model configured with hyperparameters with train data
            y_pred_ttl = list(y_pred_arr[0])
            y_pred_ttl.extend(y_pred_arr[1])
            y_pred_ttl.extend(y_pred_arr[2])
            y_pred_ttl.extend(y_pred_arr[3])
            y_pred_ttl.extend(y_pred_arr[4])

            threshold_2 = Find_Optimal_Cutoff(y_pre_train, y_pred_ttl)

            if np.mean(roc_auc_score) > bestMetricDev:
                print("\Change the best roc auc score ", bestMetricDev, " by: ", np.mean(roc_auc_score))
                bestMetricDev = np.mean(roc_auc_score)
                bestHyperparameters['nu'] = params['nu'][c]
                bestHyperparameters['threshold_1'] = np.mean(threshold_1)
                bestHyperparameters['threshold_2'] = threshold_2
                bestHyperparameters['y_pred_val'] = y_pred_ttl

        if debug:
            print("nu: ", bestHyperparameters["nu"])
            if (bestHyperparameters["nu"] == np.min(params['nu'])) or (bestHyperparameters["nu"] == np.max(params['nu'])):
                print("==============================================================")
                print("WARNING!!!!!!!")
                print("==============================================================")


        clf = svm.NuSVC(kernel="precomputed", nu=bestHyperparameters['nu'], probability=True, random_state=random_state)
        clf = clf.fit(X_pre_train, y_pre_train)
        y_pred_test = clf.predict_proba(X_test)[:,1]
        y_pred_Xtrain = clf.predict(X_pre_train)
        
        selecCalculateMetrics_aux = ['umbral: 0.5' , 
                                     'umbral: threshold_2 (datos train)']
                        
            
        for j in range(len(selecCalculateMetrics_aux)):
            
            v_accuracy_test = []
            v_accuracy_train = []
            v_specificity = []
            v_sensitivity = []
            v_precision = []
            v_recall = []
            v_f1score = []
            v_accuracy = []
            auc_score = []

            if selecCalculateMetrics_aux[j] == 'umbral: 0.5':           
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > 0.5).astype('int')
            elif selecCalculateMetrics_aux[j] == 'umbral: threshold_2 (datos train)':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > bestHyperparameters["threshold_2"]).astype('int')


            v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train = utils.calculateconfusionmatrix(y_pred, y_pre_train, y_test, v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train, 0, y_pred_Xtrain)
            
            matrix_all_values[j*8:j*8 + 8, i] = v_specificity[0], v_recall[0], v_f1score[0], v_precision[0], v_accuracy[0], v_accuracy_test[0], v_accuracy_train[0], auc_score[0]
                                 
    
    print()
    print("###################### FINAL RESULTS  - With threshold train ######################")
    print()
    
    utils.printOutAlgorithm(matrix_all_values[8,:], matrix_all_values[9,:], matrix_all_values[10,:], matrix_all_values[11,:], matrix_all_values[12,:], matrix_all_values[13,:], matrix_all_values[14,:], matrix_all_values[15,:])
    
    return matrix_all_values
    
from sklearn.linear_model import LogisticRegression


def LR(params, folders, dependencia, sigma, kfold=5):  
    matrix_all_values = np.zeros((16, len(folders)))

    for i in range(len(folders)):
        
        print()
        print("###################### RESULTS [LR] FOLDER " + folders[i] + " ######################")
        print()
        
        X_pre_train = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_train_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")
        X_test = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_test_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")

        y_pre_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_train_tensor.csv')
        y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_test_tensor.csv')

        kf = KFold(n_splits=kfold, shuffle=False)
        kf.get_n_splits(X_pre_train)

        bestHyperparameters = {'C': 0}
        
        print("Dimensions: ")
        print("X_pre_train:", X_pre_train.shape)
        print("X_test:", X_test.shape)
        print("y_pre_train:", y_pre_train.shape)
        print("y_test:", y_test.shape)
        

        bestMetricDev = 0
        for c in range(len(params['C'])):
            # Build the model with the corresponding hyperparameters
            clf = LogisticRegression(solver='liblinear', C=params['C'][c], penalty='l2', n_jobs=32)

            roc_auc_score = []
            threshold_1 = []
            threshold_2 = 0
            y_pred_arr = []
            # Divide the test set in K partitions
            for train_index, val_index in kf.split(X_pre_train):
                X_train, X_val = X_pre_train.iloc[train_index].reset_index(drop=True), X_pre_train.iloc[val_index].reset_index(drop=True)
                y_train, y_val = y_pre_train.iloc[train_index].reset_index(drop=True), y_pre_train.iloc[val_index].reset_index(drop=True)


                clf = clf.fit(np.array(X_train), np.array(y_train))
                y_pred = clf.predict_proba(X_val)[:,1]

                threshold_1.append(Find_Optimal_Cutoff(y_val, y_pred))
                auc = sklearn.metrics.roc_auc_score(y_val, y_pred)
                roc_auc_score.append(auc)

                y_pred_arr.append(y_pred)

            # Get the threshold for: model configured with hyperparameters with train data
            y_pred_ttl = list(y_pred_arr[0])
            y_pred_ttl.extend(y_pred_arr[1])
            y_pred_ttl.extend(y_pred_arr[2])
            y_pred_ttl.extend(y_pred_arr[3])
            y_pred_ttl.extend(y_pred_arr[4])

            threshold_2 = Find_Optimal_Cutoff(y_pre_train, y_pred_ttl)

            if np.mean(roc_auc_score) > bestMetricDev:
                print("\tChange the best roc auc score ", bestMetricDev, " by: ", np.mean(roc_auc_score))
                bestMetricDev = np.mean(roc_auc_score)
                bestHyperparameters['C'] = params['C'][c]
                bestHyperparameters['threshold_1'] = np.mean(threshold_1)
                bestHyperparameters['threshold_2'] = threshold_2
                bestHyperparameters['y_pred_val'] = y_pred_ttl

        
        print("Best roc auc score: ", bestMetricDev)
        print("C: ", bestHyperparameters["C"])
        print("threshold obatined with train: ", bestHyperparameters["threshold_2"])
        if (bestHyperparameters["C"] == params['C'][0]) or (bestHyperparameters["C"] == params['C'][-1]):
            print("====================================================================================")
            print("Caution!")
            print("Value of C at lower or upper extrema")
            print("====================================================================================")
        

        clf = LogisticRegression(solver='liblinear', C=bestHyperparameters['C'],  penalty='l2', n_jobs=24)
        clf = clf.fit(np.array(X_pre_train), np.array(y_pre_train))  
        y_pred_test = clf.predict_proba(X_test)[:,1]
        y_pred_Xtrain = clf.predict(X_pre_train)
        
        selecCalculateMetrics_aux = ['umbral: 0.5' , 
                                     'umbral: threshold_2 (datos train)']
                        
            
        for j in range(len(selecCalculateMetrics_aux)):
            
            v_accuracy_test = []
            v_accuracy_train = []
            v_specificity = []
            v_sensitivity = []
            v_precision = []
            v_recall = []
            v_f1score = []
            v_accuracy = []
            auc_score = []

            print("folder: " + str(folders[i]) + " | y_test.shape: " + str(y_test.shape)+ " | y_pred_test.shape: " + str(y_pred_test.shape))
            
            if selecCalculateMetrics_aux[j] == 'umbral: 0.5':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > 0.5).astype('int')
            elif selecCalculateMetrics_aux[j] == 'umbral: threshold_2 (datos train)':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > bestHyperparameters["threshold_2"]).astype('int')


            v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train = utils.calculateconfusionmatrix(y_pred, y_pre_train, y_test, v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train, 0, y_pred_Xtrain)
            
            matrix_all_values[j*8:j*8 + 8, i] = v_specificity[0], v_recall[0], v_f1score[0], v_precision[0], v_accuracy[0], v_accuracy_test[0], v_accuracy_train[0], auc_score[0]
                                 
                
    print()
    print("###################### FINAL RESULTS - With threshold train ######################")
    print()
    
    utils.printOutAlgorithm(matrix_all_values[8,:], matrix_all_values[9,:], matrix_all_values[10,:], matrix_all_values[11,:], matrix_all_values[12,:], matrix_all_values[13,:], matrix_all_values[14,:], matrix_all_values[15,:])
    
    return matrix_all_values
    
    
        
def randomForest(params, folders, dependencia, sigma, kfold=5):  
    matrix_all_values = np.zeros((16, len(folders)))

    for i in range(len(folders)):
        
        print()
        print("###################### RESULTS [random forest] FOLDER " + folders[i] + " ######################")
        print()
        
        X_pre_train = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_train_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")
        X_test = pd.read_csv("./data_exponential/" + dependencia + "/exponential/exponential_kernel_test_" + folders[i] + "_VSigma_" + str(sigma) + ".csv")

        y_pre_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_train_tensor.csv')
        y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Subconjuntos_3D/' + folders[i] + '/y_test_tensor.csv')

        kf = KFold(n_splits=kfold, shuffle=False)
        kf.get_n_splits(X_pre_train)

        bestHyperparameters = {'min_samples_leaf': 0, 'max_depth':0, 'n_estimators':0}

        bestMetricDev = 0
        for msl in range(len(params['min_samples_leaf'])):
            for md in range(len(params['max_depth'])):
                for ne in range(len(params['n_estimators'])):
                    clf = RandomForestClassifier(min_samples_leaf=params['min_samples_leaf'][msl], max_depth=params['max_depth'][md], n_estimators=params['n_estimators'][ne], n_jobs=48)

                    roc_auc_score = []
                    threshold_1 = []
                    threshold_2 = 0
                    y_pred_arr = []
                    # Divide the test set in K partitions
                    for train_index, val_index in kf.split(X_pre_train):
                        X_train, X_val = X_pre_train.iloc[train_index].reset_index(drop=True), X_pre_train.iloc[val_index].reset_index(drop=True)
                        y_train, y_val = y_pre_train.iloc[train_index].reset_index(drop=True), y_pre_train.iloc[val_index].reset_index(drop=True)


                        clf = clf.fit(np.array(X_train), np.array(y_train))
                        y_pred = clf.predict_proba(X_val)[:,1]

                        threshold_1.append(Find_Optimal_Cutoff(y_val, y_pred))
                        auc = sklearn.metrics.roc_auc_score(y_val, y_pred)
                        roc_auc_score.append(auc)

                        y_pred_arr.append(y_pred)

                    # Get the threshold for: model configured with hyperparameters with train data
                    y_pred_ttl = list(y_pred_arr[0])
                    y_pred_ttl.extend(y_pred_arr[1])
                    y_pred_ttl.extend(y_pred_arr[2])
                    y_pred_ttl.extend(y_pred_arr[3])
                    y_pred_ttl.extend(y_pred_arr[4])

                    threshold_2 = Find_Optimal_Cutoff(y_pre_train, y_pred_ttl)
                    
                    if np.mean(roc_auc_score) > bestMetricDev:
                        #print("\tChange the best roc auc score ", bestMetricDev, " by: ", np.mean(roc_auc_score))
                        bestMetricDev = np.mean(roc_auc_score)
                        bestHyperparameters['min_samples_leaf'] = params['min_samples_leaf'][msl]
                        bestHyperparameters['max_depth'] = params['max_depth'][md]
                        bestHyperparameters['n_estimators'] = params['n_estimators'][ne]
                        bestHyperparameters['threshold_1'] = np.mean(threshold_1)
                        bestHyperparameters['threshold_2'] = threshold_2
                        bestHyperparameters['y_pred_val'] = y_pred_ttl


        print("Best roc auc score: ", bestMetricDev)
        print("min_samples_leaf: ", bestHyperparameters["min_samples_leaf"])
        print("max_depth: ", bestHyperparameters["max_depth"])
        print("n_estimators: ", bestHyperparameters["n_estimators"])
        print("threshold obtained with train: ", bestHyperparameters["threshold_2"])

        if (bestHyperparameters["min_samples_leaf"] == params['min_samples_leaf'][0]) or (bestHyperparameters["min_samples_leaf"] == params['min_samples_leaf'][-1]):
            print("====================================================================================")
            print("Caution!")
            print("Value of min_samples_leaf at low end or high extrema")
            print("====================================================================================")
        if (bestHyperparameters["max_depth"] == params['max_depth'][0]) or (bestHyperparameters["max_depth"] == params['max_depth'][-1]):
            print("====================================================================================")
            print("Caution!")
            print("Value of max_depth at low end or high extrema")
            print("====================================================================================")
        if (bestHyperparameters["n_estimators"] == params['n_estimators'][0]) or (bestHyperparameters["n_estimators"] == params['n_estimators'][-1]):
            print("====================================================================================")
            print("Caution!")
            print("Value of n_estimators at low end or high extrema")
            print("====================================================================================")


        clf = RandomForestClassifier(min_samples_leaf=bestHyperparameters['min_samples_leaf'], max_depth=bestHyperparameters['max_depth'], n_estimators=bestHyperparameters['n_estimators'], n_jobs=48)
        clf = clf.fit(np.array(X_pre_train), np.array(y_pre_train))  
        y_pred_test = clf.predict_proba(X_test)[:,1]
        y_pred_Xtrain = clf.predict(X_pre_train)
        

        y_pred_Xtrain = clf.predict(X_pre_train)
        
        selecCalculateMetrics_aux = ['umbral: 0.5' , 
                                     'umbral: threshold_2 (datos train)']                        
            
        for j in range(len(selecCalculateMetrics_aux)):
            
            v_accuracy_test = []
            v_accuracy_train = []
            v_specificity = []
            v_sensitivity = []
            v_precision = []
            v_recall = []
            v_f1score = []
            v_accuracy = []
            auc_score = []

            if selecCalculateMetrics_aux[j] == 'umbral: 0.5':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > 0.5).astype('int')
            elif selecCalculateMetrics_aux[j] == 'umbral: threshold_2 (datos train)':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > bestHyperparameters["threshold_2"]).astype('int')

            v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train = utils.calculateconfusionmatrix(y_pred, y_pre_train, y_test, v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train, 0, y_pred_Xtrain)
            
            matrix_all_values[j*8:j*8 + 8, i] = v_specificity[0], v_recall[0], v_f1score[0], v_precision[0], v_accuracy[0], v_accuracy_test[0], v_accuracy_train[0], auc_score[0]
                                 
        
    print()
    print("###################### FINAL RESULTS RF ######################")
    print()
    
    print("====> WITH THRESHOLD OBTAINED WITH THE TRAIN DATA")
    print()
    utils.printOutAlgorithm(matrix_all_values[8,:], matrix_all_values[9,:], matrix_all_values[10,:], matrix_all_values[11,:], matrix_all_values[12,:], matrix_all_values[13,:], matrix_all_values[14,:], matrix_all_values[15,:])

       
    return matrix_all_values
   


## DTW_D

In [ ]:
norm = "none"
folders = ["S1", "S2", "S3", "S4", "S5"]
sigma = [0.8, 0.85, 0.9, 0.95, 1, 1.25, 1.5]
dependencia="D"

In [ ]:
for index_sigma in range(len(sigma)):
    for index_data in range(len(folders)):
        X_pre_train = pd.read_csv('../../Step1_DTW/data_generated_by_dtw/DTW_D/'+ folders[index_data] + '/X_train.csv')
        X_test = pd.read_csv('../../Step1_DTW/data_generated_by_dtw/DTW_D/'+ folders[index_data] + '/X_test.csv')
        print(X_pre_train.shape)
        print(X_test.shape)
        
        if norm == "sqrt_max":
            print("Normalization sqrt max")
            X_train_aux = np.zeros((X_pre_train.shape[0],X_pre_train.shape[1]))
            for index in range(X_pre_train.shape[0]): 
                X_train_aux[:,index] = X_pre_train.iloc[:,index] / np.sqrt(np.max(X_pre_train.iloc[:,index]))
                X_train_aux[index,:] = X_pre_train.iloc[index,:] / np.sqrt(np.max(X_pre_train.iloc[:,index]))
            X_pre_train = pd.DataFrame(X_train_aux)

            X_test_aux = np.zeros((X_test.shape[0],X_test.shape[1]))
            for index in range(X_test.shape[1]): 
                X_test_aux[:,index] = X_test.iloc[:,index] / np.sqrt(np.max(X_test.iloc[:,index]))
                X_test_aux[index,:] = X_test.iloc[index,:] / np.sqrt(np.max(X_test.iloc[index,:]))
            X_test = pd.DataFrame(X_test_aux)  
        
        else:
            print("No normalization...")

        exponential_dtw_kernel = pd.DataFrame(np.exp(-X_pre_train/(2*sigma[index_sigma]**2)))
        exponential_dtw_kernel_test = pd.DataFrame(np.exp(-X_test/(2*sigma[index_sigma]**2)))

        exponential_dtw_kernel = np.round(exponential_dtw_kernel, 6)
        exponential_dtw_kernel_test = np.round(exponential_dtw_kernel_test, 6)

        print(exponential_dtw_kernel.loc[0])

        eigenvalues = numpy.linalg.eig(exponential_dtw_kernel)
        matriz = np.array(exponential_dtw_kernel)
        if (np.sum(np.abs(eigenvalues[0])>0) == exponential_dtw_kernel.shape[0]) and (np.array_equal(matriz, matriz.T)):
            print("---------------------------")
            print("Positive eigenvalue and symmetric matrix")
            print("---------------------------")
            print("Meets kernel condition")
            print("---------------------------")
        else:
            print("==============================")
            print("NO Meets kernel condition")
            print("==============================")
            
        exponential_dtw_kernel.to_csv("./data_exponential/D/exponential/exponential_kernel_train_" + folders[index_data] + "_VSigma_" + str(sigma[index_sigma]) + ".csv", index=False)
        exponential_dtw_kernel_test.to_csv("./data_exponential/D/exponential/exponential_kernel_test_" + folders[index_data] + "_VSigma_" + str(sigma[index_sigma]) + ".csv", index=False)


In [ ]:
results_ttl_05_DTW_D = {}
results_ttl_train_DTW_D = {}

In [ ]:
# LR
C = [.0000001, .000001, .00001, .0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, .75, 1, 3, 5, 8, 10 ,12, 15]
params = {'C': C}
debug = True
bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")
        
    results = LR(params, folders, dependencia, sigma[index])
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value SIGMA: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_LR_DTW_D = LR(params, folders, dependencia, best_value_sigma)
results_ttl_05_DTW_D['LR'] = results_LR_DTW_D[[0,1,7], :]
results_ttl_train_DTW_D['LR'] = results_LR_DTW_D[[8,9,15], :]

In [ ]:
# RF
folders = ["S1", "S2", "S3", "S4", "S5"]

norm = "none"
max_depth = np.arange(10, 42, 4)
min_samples_leaf = np.array([2, 4, 9,  13,  17,  22,  43])
n_estimators = np.array([30, 50, 100, 200, 400, 600])

params = {'max_depth': max_depth,
         'min_samples_leaf': min_samples_leaf,
         'n_estimators': n_estimators}

bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")
        
    results = randomForest(params, folders, dependencia, sigma[index])
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value SIGMA: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_RF_DTW_D = randomForest(params, folders, dependencia, best_value_sigma)
results_ttl_05_DTW_D['RF'] = results_RF_DTW_D[[0,1,7], :]
results_ttl_train_DTW_D['RF'] = results_RF_DTW_D[[8,9,15], :]

In [ ]:
# nuSVM
folders = ["S1", "S2", "S3", "S4", "S5"]

norm = "none"
nu = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.75, 0.9]
params = {'nu': nu}
debug = True
random_state = 30

bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")

    results = nuSVM(params, folders, dependencia, sigma[index], random_state, debug)
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value gamma: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_nuSVM_DTW_D = nuSVM(params, folders, dependencia, best_value_sigma, random_state, debug)
results_ttl_05_DTW_D['nuSVM'] = results_nuSVM_DTW_D[[0,1,7], :]
results_ttl_train_DTW_D['nuSVM'] = results_nuSVM_DTW_D[[8,9,15], :]

## DTW_I

In [ ]:
norm = "none"
folders = ["S1", "S2", "S3", "S4", "S5"]
sigma = [5.5, 6.5, 7.5, 8.5, 9.5, 10.5]
dependencia="I"

In [ ]:
for index_sigma in range(len(sigma)):
    for index_data in range(len(folders)):
        
        X_pre_train = pd.read_csv('../../Step1_DTW/data_generated_by_dtw/DTW_I/'+ folders[index_data] + '/X_train.csv')
        X_test = pd.read_csv('../../Step1_DTW/data_generated_by_dtw/DTW_I/'+ folders[index_data] + '/X_test.csv')
        print(X_pre_train.shape)
        print(X_test.shape)
        
        if norm == "sqrt_max":
            print("Normalization sqrt max")
            X_train_aux = np.zeros((X_pre_train.shape[0],X_pre_train.shape[1]))
            for index in range(X_pre_train.shape[0]): 
                X_train_aux[:,index] = X_pre_train.iloc[:,index] / np.sqrt(np.max(X_pre_train.iloc[:,index]))
                X_train_aux[index,:] = X_pre_train.iloc[index,:] / np.sqrt(np.max(X_pre_train.iloc[:,index]))
            X_pre_train = pd.DataFrame(X_train_aux)

            X_test_aux = np.zeros((X_test.shape[0],X_test.shape[1]))
            for index in range(X_test.shape[1]): 
                X_test_aux[:,index] = X_test.iloc[:,index] / np.sqrt(np.max(X_test.iloc[:,index]))
                X_test_aux[index,:] = X_test.iloc[index,:] / np.sqrt(np.max(X_test.iloc[index,:]))
            X_test = pd.DataFrame(X_test_aux)  
        
        else:
            print("No normalization...")

        exponential_dtw_kernel = pd.DataFrame(np.exp(-X_pre_train/(2*sigma[index_sigma]**2)))
        exponential_dtw_kernel_test = pd.DataFrame(np.exp(-X_test/(2*sigma[index_sigma]**2)))

        exponential_dtw_kernel = np.round(exponential_dtw_kernel, 6)
        exponential_dtw_kernel_test = np.round(exponential_dtw_kernel_test, 6)

        print(exponential_dtw_kernel.loc[0])

        eigenvalues = numpy.linalg.eig(exponential_dtw_kernel)
        matriz = np.array(exponential_dtw_kernel)
        if (np.sum(np.abs(eigenvalues[0])>0) == exponential_dtw_kernel.shape[0]) and (np.array_equal(matriz, matriz.T)):
            print("---------------------------")
            print("Positive eigenvalue and symmetric matrix")
            print("---------------------------")
            print("Meets kernel condition")
            print("---------------------------")
        else:
            print("==============================")
            print("NO Meets kernel condition")
            print("==============================")
            
        exponential_dtw_kernel.to_csv("./data_exponential/I/exponential/exponential_kernel_train_" + folders[index_data] + "_VSigma_" + str(sigma[index_sigma]) + ".csv", index=False)
        exponential_dtw_kernel_test.to_csv("./data_exponential/I/exponential/exponential_kernel_test_" + folders[index_data] + "_VSigma_" + str(sigma[index_sigma]) + ".csv", index=False)


In [ ]:
results_ttl_05_DTW_I = {}
results_ttl_train_DTW_I = {}

In [ ]:
# LR
C = [.0000001, .000001, .00001, .0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, .75, 1, 3, 5, 8, 10 ,12, 15]
params = {'C': C}

bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")
        
    results = LR(params, folders, dependencia, sigma[index])
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value SIGMA: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_LR_DTW_I = LR(params, folders, dependencia, best_value_sigma)
results_ttl_05_DTW_I['LR'] = results_LR_DTW_I[[0,1,7], :]
results_ttl_train_DTW_I['LR'] = results_LR_DTW_I[[8,9,15], :]

In [ ]:
# RF
max_depth = np.arange(10, 42, 4)
min_samples_leaf = np.array([2, 4, 9,  13,  17,  22,  43])
n_estimators = np.array([30, 50, 100, 200, 400, 600])

params = {'max_depth': max_depth,
         'min_samples_leaf': min_samples_leaf,
         'n_estimators': n_estimators}

bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")
        
    results = randomForest(params, folders, dependencia, sigma[index])
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value SIGMA: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_RF_DTW_I = randomForest(params, folders, dependencia, best_value_sigma)
results_ttl_05_DTW_I['RF'] = results_RF_DTW_I[[0,1,7], :]
results_ttl_train_DTW_I['RF'] = results_RF_DTW_I[[8,9,15], :]

In [ ]:
# nuSVM
nu = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.75, 0.9]
params = {'nu': nu}
debug = True
random_state = 30


bestROC = 0
best_value_sigma = 0
for index in range(len(sigma)):
    print("================================================================================================")
    print(sigma[index])
    print("================================================================================================")

    results = nuSVM(params, folders, dependencia, sigma[index], random_state, debug)
    if bestROC < np.mean(results[7, :]):
        bestROC = np.mean(results[7, :])
        best_value_sigma = sigma[index]

print()
print("Best roc: ", bestROC, " - value gamma: ", best_value_sigma)   
print("\n\n\n")
print("BEST COMBINATION OF PARAMETERS")
print("===============================================")
results_nuSVM_DTW_I = nuSVM(params, folders, dependencia, best_value_sigma, random_state, debug) 
results_ttl_05_DTW_I['nuSVM'] = results_nuSVM_DTW_I[[0,1,7], :]
results_ttl_train_DTW_I['nuSVM'] = results_nuSVM_DTW_I[[8,9,15], :]

##  Save the results

In [ ]:
def saveData(results_ttl, name):
    print(list(results_ttl.keys()))
    values = list(results_ttl.values())
    arr = []
    for i in range(len(values)):
        for j in range(len(values[i])):
            arr.append(values[i][j])

    df_results_ttl_AE = pd.DataFrame(arr)
    print(df_results_ttl_AE.shape)
    # rows: specificity, sensitivity and roc-auc
    # columns: folders
    df_results_ttl_AE.to_excel(name)

In [ ]:
saveData(results_ttl_05_DTW_D, '../results_kernels/DTW/D/exponential_05.xlsx')

saveData(results_ttl_05_DTW_I, '../results_kernels/DTW/I/exponential_05.xlsx')

saveData(results_ttl_train_DTW_D, '../results_kernels/DTW/D/exponential_train.xlsx')

saveData(results_ttl_train_DTW_I, '../results_kernels/DTW/I/exponential_train.xlsx')